In [1]:
class ChlSouthern:
    '''
    class Chlsurf_SO_comp(runname,resultpath,savepath,meshpath,ncfileJohnson2013,first_year,last_year,
                 savefig=False, verbose=False, output=False, 
                            plotting=True, Taylor=True)
                 
    pathJohnson2013: path to numpy output files of Johnson2013 mean data 
    folder should contain:
    -Johnson2013_MEAN_1x1_Chl_mg_m3.npy
    assuming 1x1 grid as 
    ilat         = np.arange(-89.5,-29.5,1.)
    ilon         = np.arange(-179.5,180.,1.)
    
    n_levels = 1: number of mesh levels used for FESOM surface mean
    '''
    
    def __init__(self,resultpath,savepath,mesh,ncfileJohnson2013,first_year,last_year,
                 savefig=False,
                 n_levels = 1,
                 verbose=False,
                 plotting=True,
                 output=False,
                 Taylor=True,
                 runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.n_levels = n_levels
        self.ncfileJohnson2013 = ncfileJohnson2013
        self.verbose = verbose
        self.plotting = plotting
        self.output = output
        self.Taylor = Taylor

        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import cartopy.feature as cfeature
        import pyfesom2 as pf
        from Py_f2recom_toolbox import plt_Taylor_norm
        
        mapproj = pf.get_proj('sp')
        
        if(self.verbose):
            print('Processing {0}'.format(self.resultpath))
        
        # load Johnson2013 CHl.a data -------------------------------------------------------------------------------------
        chlJohnson = np.load(self.ncfileJohnson2013+'')
        
        #lat = np.load(pathJohnson2013+'Johnson2013_lat.npy')
        #lon = np.load(pathJohnson2013+'Johnson2013_lon.npy')
        
        
        chlJohnson_log10 = np.log10(chlJohnson)
        
        chlJohnsonlabel = 'Johnson et al. (1998-2019)'
        chlJohnsonunit = 'Chl.a [mg Chl m$^{-3}$]'
                
        # define regular mesh -------------------------------------------------------------------------------------
        lat = np.arange(-89.5,90,1)
        lat_SO = np.arange(-89.5,-29.5,1.)
        lon = np.arange(-179.5,180.,1.)
        
        latdic, londic = np.meshgrid(lat, lon)
        
        latdic_SO, londic_SO = np.meshgrid(lat_SO, lon)
        
        # load FESOM mesh -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(self.meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)
        
        lon_fesom = mesh.x2
        lat_fesom = mesh.y2        
        
        # surface: mean over n mesh levels
        #self.n_levels
        f_depth = mesh.zlev[self.n_levels]
        if(self.verbose):
            print('***\nUsing upper {0} layers to depth {1} m for surface FESOM data!\n***'.format(
                n_levels,f_depth))
        
        # load FESOM Nanophyto Chl.a data -------------------------------------------------------------------------------------        
        #ncFESOMChl = self.resultpath + '/PhyChl.fesom.1948.nc'
        #!ncdump -h $ncFESOMChl
        
        ChlAfesom = pf.get_data(self.resultpath, "PhyChl", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
        
        
        ChlAfesom_surf = np.nanmean(ChlAfesom[:,:n_levels],axis=1)
        
        # mean over whole water column
        #ChlAfesom_mean = np.nanmean(ChlAfesom, axis=1)
        
        labelfesomNano = 'FESOM Nanophyto Chl.a {0}-{1}'.format(self.fyear,self.lyear)        
        #unitfesomNano = 'Chl.a [mmol m$^{-3}$]' 

        # load FESOM Diatom Chl.a data -------------------------------------------------------------------------------------
        
        #ncFESOMDiaChl = self.resultpath + '/DiaChl.fesom.1948.nc'
        #!ncdump -h $ncFESOMDiaChl
        
        DiaChlfesom = pf.get_data(self.resultpath, "DiaChl", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
        
        labelfesomDia = 'FESOM Diatom Chl.a {0}-{1}'.format(self.fyear,self.lyear)
        
        # surface: mean over n mesh levels
        DiaChlfesom_surf = np.nanmean(DiaChlfesom[:,:n_levels],axis=1)
    
        unitfesom = 'Chl.a [mg m$^{-3}$]'
        labelfesom = 'FESOM ({0}-{1})'.format(self.fyear,self.lyear)
        
        
        # load FESOM Coccolithophore Chl.a data -------------------------------------------------------------------------------------

        from pathlib import Path
        cocco_path = Path(self.resultpath + '/CoccoChl.fesom.'+str(years[0])+'.nc') # assuming that coccos were used for the entire simulation if they were used in the first year of simulation
        phaeo_path = Path(self.resultpath + '/PhaeoChl.fesom.'+str(years[0])+'.nc') # assuming that phaeo was used for the entire simulation if they were used in the first year of simulation
        
        if cocco_path.is_file():
            
            CoccoChlfesom = pf.get_data(self.resultpath, "CoccoChl", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)
            
            labelfesomCocco = 'FESOM-REcoM Cocco Chl.a {0}-{1}'.format(self.fyear,self.lyear)
            
            CoccoChlfesom_surf = np.nanmean(CoccoChlfesom[:,:n_levels],axis=1)

            # load FESOM Phaeo Chl.a data ---------------------------------------------------------------------------------------

            if phaeo_path.is_file():
            
                PhaeoChlfesom = pf.get_data(self.resultpath, "PhaeoChl", years, mesh, 
                                   how="mean", compute=True, runid=self.runname, silent=True)
                
                labelfesomPhaeo = 'FESOM-REcoM Phaeo Chl.a {0}-{1}'.format(self.fyear,self.lyear)
                
                PhaeoChlfesom_surf = PhaeoChlfesom[:,0]
                
                print('4-phytoplankton model is used')

            else:
            
                print('3-phytoplankton model is used')
        
        else:
            
            print('2-phytoplankton model is used')
        
        # interpolate FESOM CHl.a to regular -------------------------------------------------------------------------------------
        ChlAfesom_surf_interp = pf.fesom2regular(
                data = ChlAfesom_surf,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
        DiaChlfesom_surf_interp = pf.fesom2regular(
                data = DiaChlfesom_surf,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
        if cocco_path.is_file():
            CoccoChlfesom_surf_interp = pf.fesom2regular(
                data = CoccoChlfesom_surf,
                mesh = mesh,
                lons = londic, 
                lats = latdic)

        if phaeo_path.is_file():
            PhaeoChlfesom_surf_interp = pf.fesom2regular(
                data = PhaeoChlfesom_surf,
                mesh = mesh,
                lons = londic, 
                lats = latdic)
        
         
        # Nanophyto + Diatoms (+ Coccos): TOTAL CHLOROPHYLL -------------------------------------------------------------------------------------
        ChlAfesom_surf_interp_log10 = np.log10(ChlAfesom_surf_interp)
        DiaChlfesom_surf_interp_log10 = np.log10(DiaChlfesom_surf_interp)
        if cocco_path.is_file():
            CoccoChlfesom_surf_interp_log10 = np.log10(CoccoChlfesom_surf_interp)
        if phaeo_path.is_file():
            PhaeoChlfesom_surf_interp_log10 = np.log10(PhaeoChlfesom_surf_interp)
        
        Chl_total = ChlAfesom_surf_interp + DiaChlfesom_surf_interp
        if cocco_path.is_file():
            Chl_total = Chl_total + CoccoChlfesom_surf_interp
        if phaeo_path.is_file():
            Chl_total = Chl_total + PhaeoChlfesom_surf_interp
        Chl_total_log10 = np.log10(Chl_total)
        
        # cut to Southern Ocean -------------------------------------------------------------------------------------
        ilat = np.where(lat < -30)
        
        Chl_total_SO = np.squeeze(Chl_total[:,ilat])
        Chl_total_SO_log10 = np.squeeze(Chl_total_log10[:,ilat]) 
        
        #print(np.shape(Chl_total_SO))
        
        if False: # interpolation check
            Chl_total_preinterp = ChlAfesom_surf + DiaChlfesom_surf + CoccoChlfesom_surf + PhaeoChlfesom_surf
            
            print('\nFESOM interpolation check:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
                np.nanmin(Chl_total_preinterp),np.nanmax(Chl_total_preinterp),np.nanmean(Chl_total_preinterp),
                np.nanmin(Chl_total),np.nanmax(Chl_total),np.nanmean(Chl_total)))
        
            fig = plt.figure(figsize=(20,10))
            ax1 = plt.subplot(projection = ccrs.PlateCarree())

            m1 = plt.pcolormesh(londic_SO, latdic_SO, Chl_total_SO, 
                transform = ccrs.PlateCarree(),
                norm=colors.LogNorm(vmin=np.nanmin(Chl_total_SO), 
                                    vmax=np.nanmax(Chl_total_SO)),
                cmap='viridis')

            ax1.coastlines(resolution='110m', color='black', linewidth=1)

            cbar = fig.colorbar(m1,orientation = 'horizontal',fraction=0.1, pad=0.1, extend = 'max') 
            cbar.set_label('Interpolated '+unitfesom, fontsize=20)

        
        # apply sea mask to Johnson as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        #chlJohnson_ma = np.copy(chlJohnson)
        #chlJohnson_ma[~np.isfinite(ChlAfesom_surf_interp)] = np.nan
        
        #chlJohnson_ma_log10 = np.log10(chlJohnson_ma)
        
        # check CHl.a data -------------------------------------------------------------------------------------
        if(self.verbose):
            print('\nChl.a\nJohnson2013 min = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f} (Mean over 0 to {4} m)'.format(
                    np.nanmin(Chl_total_SO),np.nanmax(Chl_total_SO),
                    np.nanmin(chlJohnson),np.nanmax(chlJohnson),
                    f_depth))

            print('\nlog10(Chl.a)\nJohnson2013 min = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f} (Mean over 0 to {4} m)'.format(
                    np.nanmin(Chl_total_SO_log10),np.nanmax(Chl_total_SO_log10),
                    np.nanmin(chlJohnson_log10),np.nanmax(chlJohnson_log10),
                    f_depth))
        
        if(self.plotting):
            # plot each CHl.a dataset -------------------------------------------------------------------------------------        
            levels = np.array([0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
                               0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                               1,2,3,4,5,7])
            ticks = [0,0.01,0.03,0.05,0.07,0.1,0.3,0.5,0.7,1,3,5,7]
            ticks_label = ['0','0.01','0.03','0.05','0.07','0.1','0.3','0.5','0.7','1','3','5','7']

            def mygrid(m,grid=False):
                #m.coastlines(resolution='110m', color='black', linewidth=1)
                m.add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='gray')
                if grid:
                    g1 = m.gridlines(draw_labels = True)
                    g1.xlabels_top = False
                    g1.xlabel_style = {'size': 16, 'color': 'gray'}
                    g1.ylabel_style = {'size': 16, 'color': 'gray'}


            # plot Johnson2013 and SUM CHl.a data -------------------------------------------------------------------------------------            
            fig, axes = plt.subplots(1,3, 
                                     subplot_kw=dict(projection=mapproj),
                                     gridspec_kw={'hspace': 0.01, 'wspace': 0.1},
                                     figsize=(20,7), constrained_layout=False)
            # Johnson
            m1 = axes[1]
            f1 = m1.pcolormesh(londic_SO, latdic_SO, chlJohnson, 
                               transform = ccrs.PlateCarree(),
                               norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                               #vmin=1e-3,vmax=5e3)
            mygrid(m1)
            m1.set_title(chlJohnsonlabel, fontsize=16)


            # FESOM
            m2 = axes[0]
            f2 = m2.pcolormesh(londic_SO, latdic_SO, Chl_total_SO, 
                               transform = ccrs.PlateCarree(),
                               norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
            mygrid(m2)
            m2.set_title('{0}\n(0 to {1} m)'.format(labelfesom,f_depth), fontsize=16)

            # add one colorbar for first row plots below figure
            cbar = fig.colorbar(f1,
                                ax = axes[:2], 
                                location ='bottom',
                                #orientation = 'horizontal',
                                ticks = ticks,
                                fraction=0.046, pad=0.04) 
            cbar.ax.set_xticklabels(ticks_label, fontsize=16) 
            cbar.set_label('Chl.a [mg m$^{-3}$]', fontsize=16)
            cbar.ax.set_xticklabels(cbar.ax.get_xticklabels(), rotation=45)
            # Johnson - FESOM
            levels_diff = np.arange(-3,3,0.125)
            m3 = axes[2]
            f3 = m3.pcolormesh(londic_SO, latdic_SO, Chl_total_SO - chlJohnson, 
                               transform = ccrs.PlateCarree(),
                               cmap = 'RdBu',
                               norm=colors.BoundaryNorm(boundaries=levels_diff, ncolors=256)
                               )
            mygrid(m3,grid=False)
            m3.set_title('FESOM - Johnson et al.(2013)', fontsize=16)
            
            m1.text(-0.12, 1.05, 'B', transform=m1.transAxes,
                        size=30, weight='bold')
            m2.text(-0.12, 1.05, 'A', transform=m2.transAxes,
                        size=30, weight='bold')
            m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                        size=30, weight='bold')
            
            # add one colorbar for difference plot below figure
            cbar = fig.colorbar(f3,
                            ax = axes[2], 
                            orientation = 'horizontal',
                            #location ='bottom',
                            ticks = [-3,-2,-1,0,1,2,3],
                            extend = 'both',
                            fraction=0.046, pad=0.04) 
            cbar.ax.tick_params(labelsize=14)
            cbar.set_label('Chl.a [mg m$^{-3}$]', fontsize=16)
            cbar.ax.set_xticklabels(cbar.ax.get_xticklabels(), rotation=45)
            
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'Chla_Johnson'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'Chla_Johnson'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                        bbox_inches='tight')
            plt.show(block=False)  

        if(self.Taylor):
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(chlJohnson_log10)): print('WARNING: Johnson field contains NaNs')
            if np.isnan(np.min(Chl_total_SO_log10)): print('WARNING: FESOM field contains NaNs')

            # get statistics only from valid OCCCI gridpoints 
            ind_stat = np.where(np.isfinite(chlJohnson_log10))

            title = 'log10 surface Chlorophyll'
            print('\nStatistics for '+title)
            plt_Taylor_norm(chlJohnson_log10[ind_stat],Chl_total_SO_log10[ind_stat],
                                    mask=True,title=title)

            
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):                
                plt.savefig(self.savepath+self.runname+'_'+'Chla_Johnson_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'Chla_Johnson_Taylor'+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                        bbox_inches='tight')
            plt.show(block=False)
            
        if(self.output):
            self.lon = londic_SO
            self.lat = latdic_SO
            self.Chl_johnson = chlJohnson
            self.Chl_fesom = Chl_total_SO